# Demonstração do conceito de separabilidade linear vs não linear

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Criar dados de exemplo com duas features
np.random.seed(42)
X = np.random.randn(100, 2)
y = np.zeros(100)
y[X[:, 0] + X[:, 1] > 0] = 1  # Classificação linear

# Treinar um classificador linear (Regressão Logística)
clf_linear = LogisticRegression()
clf_linear.fit(X, y)

# Treinar um classificador baseado em árvore de decisão
clf_tree = DecisionTreeClassifier()
clf_tree.fit(X, y)

# Plotar os dados e as fronteiras de decisão dos classificadores
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='k')
plt.title('Fronteira de Decisão Linear')
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()
xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 50),
                     np.linspace(ylim[0], ylim[1], 50))
Z = clf_linear.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.4, cmap='coolwarm')

plt.subplot(1, 2, 2)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='k')
plt.title('Fronteira de Decisão de Árvore de Decisão')
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()
xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 50),
                     np.linspace(ylim[0], ylim[1], 50))
Z = clf_tree.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, alpha=0.4, cmap='coolwarm')

plt.tight_layout()
plt.show()

# Pré-processamento - Transformação dos datasets de treinamento e teste

Importação de pacotes

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score

Carregamento de dataset, EDA básica e exclusão de colunas indesejadas

In [ ]:
# Carregar o conjunto de dados Titanic
titanic_data = pd.read_csv('train.csv')

# Vamos checar a qtd de valores únicos e a proporção de nulos por coluna
for col in titanic_data.columns:
  qtd_unique = len(titanic_data[col].unique())
  perc_nulls = titanic_data[col].isna().mean()
  print(f"{col}: {qtd_unique} unique values, and {100*perc_nulls:.2f}% of nulls")

# Vamos remover algumas colunas que não serão úteis para este exemplo
titanic_data = titanic_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

PassengerId: 891 unique values, and 0.00% of nulls
Survived: 2 unique values, and 0.00% of nulls
Pclass: 3 unique values, and 0.00% of nulls
Name: 891 unique values, and 0.00% of nulls
Sex: 2 unique values, and 0.00% of nulls
Age: 89 unique values, and 19.87% of nulls
SibSp: 7 unique values, and 0.00% of nulls
Parch: 7 unique values, and 0.00% of nulls
Ticket: 681 unique values, and 0.00% of nulls
Fare: 248 unique values, and 0.00% of nulls
Cabin: 148 unique values, and 77.10% of nulls
Embarked: 4 unique values, and 0.22% of nulls


Transformação dos datasets de treinamento e teste

In [ ]:
# Vamos definir uma função que encapsulará as nossas rotinas de transformação de datasets
def transform_dataset(df, num_imputer, cat_imputer, onehot_encoder, num_variables, cat_variables):

  # Aplicação do numerical imputer
  df[num_variables] = num_imputer.transform(df[num_variables])

  # Aplicação do categorical imputer
  df[cat_variables] = cat_imputer.transform(df[cat_variables])

  # Aplicação do one_hot encoder
  dummy_variables_df = pd.DataFrame(
    data = onehot_encoder.transform(df[cat_variables]).toarray(),
    columns = onehot_encoder.get_feature_names_out(),
    index = df.index
  )
  df = pd.concat([
    df.drop(columns = cat_variables),
    dummy_variables_df
  ], axis = 1)

  return df

In [ ]:
# Definindo colunas numericas
numerical_original_columns = ['Age', 'Fare']

# Definindo colunas categóricas
categorical_original_columns = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']

# Separando as features e o target
X = titanic_data.drop(columns = ['Survived'])
y = titanic_data['Survived']

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustando imputers para valores numéricos ausentes com base no conjunto de treinamento
numerical_imputer = SimpleImputer(strategy='mean') # imputaremos os nulos com a média da coluna
numerical_imputer.fit(X_train[numerical_original_columns])

# Ajustando imputers para valores categóricos ausentes com base no conjunto de treinamento
categorical_imputer = SimpleImputer(strategy='most_frequent')  # imputaremos os nulos com a moda da coluna
categorical_imputer.fit(X_train[categorical_original_columns])

# Ajustando um transformador de varáveis categóricas com base no conjunto de treinamento
one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(X_train[categorical_original_columns])

# Transformando o dataset de treinamento de acordo com os objetos ajustados anteriormente
X_train = transform_dataset(
  X_train, numerical_imputer, categorical_imputer, one_hot_encoder,
  numerical_original_columns, categorical_original_columns
)

# Transformando o dataset de teste de acordo com os objetos ajustados em treinamento
X_test = transform_dataset(
  X_test, numerical_imputer, categorical_imputer, one_hot_encoder,
  numerical_original_columns, categorical_original_columns
)

# Decision Tree Classifier com hiperparâmetros default

In [ ]:
# Treinando o modelo decision_tree
dec_tree_def_clf = DecisionTreeClassifier()
dec_tree_def_clf.fit(X_train, y_train)

# Predizendo saídas para o dataset de treinamento
y_train_pred = dec_tree_def_clf.predict(X_train)

# Predizendo saídas para o dataset de testes
y_test_pred = dec_tree_def_clf.predict(X_test)

# Calculando AUC em treinamento e em teste
roc_auc_train = roc_auc_score(y_train, y_train_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print(f"AUC_train = {roc_auc_train}, AUC_tes = {roc_auc_test}")

AUC_train = 0.9746201425305903, AUC_tes = 0.7696267696267697


# Decision Tree Classifier com hiperparâmetros otimizados via grid-search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definindo o grid de parâmetros a serem testados
param_grid = {
  'criterion': ['gini', 'entropy'],
  'max_depth': [None, 5, 10, 15],
  'min_samples_split': [2, 5, 10],
  'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
  estimator = DecisionTreeClassifier(),
  param_grid = param_grid,
  cv = 5
)

# Ajustando o GridSearch ao conjunto de treinamento
grid_search.fit(X_train, y_train)

# Obtendo os melhores parâmetros encontrados
dec_tree_gs_best_params = grid_search.best_params_
print(dec_tree_gs_best_params)

{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}


In [ ]:
# Treinando o modelo
dec_tree_gs_clf = DecisionTreeClassifier(**dec_tree_gs_best_params)
dec_tree_gs_clf.fit(X_train, y_train)

# Predizendo saídas para o dataset de treinamento
y_train_pred = dec_tree_gs_clf.predict(X_train)

# Predizendo saídas para o dataset de testes
y_test_pred = dec_tree_gs_clf.predict(X_test)

# Calculando AUC em treinamento e em teste
roc_auc_train = roc_auc_score(y_train, y_train_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print(f"AUC_train = {roc_auc_train}, AUC_tes = {roc_auc_test}")

AUC_train = 0.8021379588543768, AUC_tes = 0.7834620334620336


# Decision Tree Classifier com hiperparâmetros otimizados via random-search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Definindo o espaço de busca para os hiperparâmetros
param_dist = {
  'criterion': ['gini', 'entropy'],
  'max_depth': randint(1, 20),
  'min_samples_split': randint(2, 20),
  'min_samples_leaf': randint(1, 10)
}

random_search = RandomizedSearchCV(
  estimator = DecisionTreeClassifier(),
  param_distributions = param_dist,
  n_iter = 200,
  cv = 5
)

# Ajustando a pesquisa aleatória ao conjunto de treinamento
random_search.fit(X_train, y_train)

# Obtendo os melhores parâmetros encontrados
dec_tree_rs_best_params = random_search.best_params_
print(dec_tree_rs_best_params)

{'criterion': 'entropy', 'max_depth': 3, 'min_samples_leaf': 4, 'min_samples_split': 10}


In [ ]:
# Treinando o modelo
dec_tree_rs_clf = DecisionTreeClassifier(**dec_tree_rs_best_params)
dec_tree_rs_clf.fit(X_train, y_train)

# Predizendo saídas para o dataset de treinamento
y_train_pred = dec_tree_rs_clf.predict(X_train)

# Predizendo saídas para o dataset de testes
y_test_pred = dec_tree_rs_clf.predict(X_test)

# Calculando AUC em treinamento e em teste
roc_auc_train = roc_auc_score(y_train, y_train_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print(f"AUC_train = {roc_auc_train}, AUC_tes = {roc_auc_test}")

AUC_train = 0.8101384967056608, AUC_tes = 0.7779279279279279


# Random Forest Classifier com hiperparâmetros otimizados via random-search

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Definindo o espaço de busca para os hiperparâmetros
param_dist = {
    'n_estimators': randint(10, 200),
    'criterion': ['gini', 'entropy'],
    'max_depth': [None] + list(randint(1, 20).rvs(5)),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None]
}

random_search = RandomizedSearchCV(
  estimator=RandomForestClassifier(),
  param_distributions=param_dist,
  n_iter=200,
  cv=5,
  random_state=42
)

# Ajustando a pesquisa aleatória ao conjunto de treinamento
random_search.fit(X_train, y_train)

# Obtendo os melhores parâmetros encontrados
rforest_rs_best_params = random_search.best_params_
print(rforest_rs_best_params)

{'criterion': 'entropy', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 6, 'min_samples_split': 3, 'n_estimators': 136}


In [ ]:
# Treinando o modelo
rforest_rs_clf = RandomForestClassifier(**rforest_rs_best_params)
rforest_rs_clf.fit(X_train, y_train)

# Predizendo saídas para o dataset de treinamento
y_train_pred = rforest_rs_clf.predict(X_train)

# Predizendo saídas para o dataset de testes
y_test_pred = rforest_rs_clf.predict(X_test)

# Calculando AUC em treinamento e em teste
roc_auc_train = roc_auc_score(y_train, y_train_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print(f"AUC_train = {roc_auc_train}, AUC_tes = {roc_auc_test}")

AUC_train = 0.8214669893774371, AUC_tes = 0.7671814671814672


# KNN com hiperparâmetros otimizados via random-search

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.neighbors import KNeighborsClassifier

# Definindo o espaço de busca para os hiperparâmetros
param_dist = {
    'n_neighbors': randint(1, 50),
    'weights': ['uniform', 'distance'],
    'p': [1, 2]  # para a distância de Minkowski (1 para Manhattan, 2 para Euclidiana)
}

random_search = RandomizedSearchCV(
  estimator = KNeighborsClassifier(),
  param_distributions = param_dist,
  n_iter = 200,
  cv = 5
)

# Ajustando a pesquisa aleatória ao conjunto de treinamento
random_search.fit(X_train, y_train)

# Obtendo os melhores parâmetros encontrados
knn_rs_best_params = random_search.best_params_
print(knn_rs_best_params)

{'n_neighbors': 15, 'p': 1, 'weights': 'distance'}


In [ ]:
# Treinando o modelo
knn_rs_clf = KNeighborsClassifier(**knn_rs_best_params)
knn_rs_clf.fit(X_train, y_train)

# Predizendo saídas para o dataset de treinamento
y_train_pred = knn_rs_clf.predict(X_train)

# Predizendo saídas para o dataset de testes
y_test_pred = knn_rs_clf.predict(X_test)

# Calculando AUC em treinamento e em teste
roc_auc_train = roc_auc_score(y_train, y_train_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print(f"AUC_train = {roc_auc_train}, AUC_tes = {roc_auc_test}")

AUC_train = 0.9746201425305903, AUC_tes = 0.7393822393822393


# Cross-validation com Pipelines

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer

# Definindo colunas numericas
numerical_original_columns = ['Age', 'Fare']

# Definindo colunas categóricas
categorical_original_columns = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']

# Separando as features e o target
X = titanic_data.drop(columns=['Survived'])
y = titanic_data['Survived']

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando pipeline para pré-processamento e classificação
pipeline = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('numeric', SimpleImputer(strategy='mean'), numerical_original_columns),
        ('categorical', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(categories='auto', handle_unknown='ignore'))
        ]), categorical_original_columns)
    ])),
    ('classifier', DecisionTreeClassifier())
])

# Treinando o modelo e avaliando usando cross-validation
scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')

# Ajustando o modelo final usando todos os dados de treinamento
pipeline.fit(X_train, y_train)

# Avaliando o modelo no conjunto de teste
roc_auc_test = roc_auc_score(y_test, pipeline.predict(X_test))

print(f"AUC_train (CV) = {scores.mean()}, AUC_test = {roc_auc_test}")

AUC_train (CV) = 0.7584457795725401, AUC_test = 0.7513513513513514


# Cross-validation com Pipelines incluindo random search

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint
from sklearn.metrics import roc_auc_score
import pandas as pd

# Definindo colunas numericas
numerical_original_columns = ['Age', 'Fare']

# Definindo colunas categóricas
categorical_original_columns = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']

# Separando as features e o target
X = titanic_data.drop(columns=['Survived'])
y = titanic_data['Survived']

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando pipeline para pré-processamento e classificação
pipeline = Pipeline([
    ('preprocessor', ColumnTransformer([
        ('numeric', SimpleImputer(strategy='mean'), numerical_original_columns),
        ('categorical', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(categories='auto', handle_unknown='ignore'))
        ]), categorical_original_columns)
    ])),
    ('classifier', DecisionTreeClassifier())
])

# Definindo o espaço de busca de hiperparâmetros
param_dist = {
    'classifier__max_depth': randint(1, 20),  # Profundidade máxima da árvore
    'classifier__min_samples_split': randint(2, 20),  # Número mínimo de amostras necessárias para dividir um nó interno
    'classifier__min_samples_leaf': randint(1, 20),  # Número mínimo de amostras necessárias para estar em um nó folha
}

# Criando o objeto RandomizedSearchCV
random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist, n_iter=50, cv=5, scoring='accuracy')

# Executando a busca aleatória de hiperparâmetros
random_search.fit(X_train, y_train)

# Melhores hiperparâmetros encontrados
best_params = random_search.best_params_

# Melhor estimador encontrado
best_estimator = random_search.best_estimator_

# Avaliando o modelo no conjunto de teste usando os melhores hiperparâmetros encontrados
roc_auc_test = roc_auc_score(y_test, best_estimator.predict(X_test))

print(f"Best parameters: {best_params}")
print(f"AUC_train (CV) = {random_search.best_score_}, AUC_test = {roc_auc_test}")


Best parameters: {'classifier__max_depth': 9, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 16}
AUC_train (CV) = 0.8159854230276766, AUC_test = 0.8084942084942084
